# California Housing Dataset

In [79]:
import numpy as np

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

random_seed = 42
torch.manual_seed(random_seed)

In [80]:
# Load the dataset
X, y = fetch_california_housing(return_X_y=True)
X.shape, y.shape

((20640, 8), (20640,))

In [81]:
# Split the dataset, 20% for testing, with random seed
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((16512, 8), (4128, 8), (16512,), (4128,))

In [82]:
# Standardize the data using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

last_test = X_test[-1]

X_train.shape, X_test.shape, np.min(last_test), np.max(last_test)

((16512, 8),
 (4128, 8),
 np.float64(-0.9211376319711084),
 np.float64(0.6044549279675977))

In [83]:
# Convert the data to PyTorch tensors
X_train_tensor = torch.from_numpy(X_train).float()
y_train_tensor = torch.from_numpy(y_train).float().reshape(-1, 1)
X_test_tensor = torch.from_numpy(X_test).float()
y_test_tensor = torch.from_numpy(y_test).float().reshape(-1, 1)

X_train_tensor.shape, y_train_tensor.shape, X_test_tensor.shape, y_test_tensor.shape

(torch.Size([16512, 8]),
 torch.Size([16512, 1]),
 torch.Size([4128, 8]),
 torch.Size([4128, 1]))

In [84]:
# Defining the model, RegresionANN
class RegressionANN(nn.Module):
    def __init__(self, input_size=8, hidden_size=16, output_size=1):
        super(RegressionANN, self).__init__()
        self.input_layer = nn.Linear(input_size, hidden_size)
        self.hidden1 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = self.input_layer(x)
        x = F.relu(self.hidden1(x))
        return x

In [85]:
# Instantiate the model
model = RegressionANN()
w1, b1, w2, b2 = list(model.parameters())
b2

Parameter containing:
tensor([0.2272], requires_grad=True)

In [86]:
# Initialize loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [87]:
# Running a forward pass
y_pred = model(X_train_tensor)
loss = criterion(y_pred, y_train_tensor)
loss

tensor(4.4983, grad_fn=<MseLossBackward0>)

In [88]:
# Training the model for 100 epochs
epochs = 100
for epoch in range(1,epochs+1):
    optimizer.zero_grad() # Zero the gradients
    y_pred = model(X_train_tensor) # Forward pass
    loss = criterion(y_pred, y_train_tensor) # Compute the loss
    loss.backward() # Backward pass
    optimizer.step() # Update the weights
    if epoch % 10 == 0:
        print(f'Epoch: {epoch}, Loss: {loss.detach().item()}')

Epoch: 10, Loss: 2.22469425201416
Epoch: 20, Loss: 0.7730512619018555
Epoch: 30, Loss: 0.595940887928009
Epoch: 40, Loss: 0.5444602370262146
Epoch: 50, Loss: 0.4821970760822296
Epoch: 60, Loss: 0.4839205741882324
Epoch: 70, Loss: 0.47262129187583923
Epoch: 80, Loss: 0.4694801867008209
Epoch: 90, Loss: 0.46595755219459534
Epoch: 100, Loss: 0.46388015151023865


In [89]:
# Changing hidden layer size to 64
model = RegressionANN(hidden_size=64)
w1, b1, w2, b2 = list(model.parameters())

In [90]:
# Training the model for 100 epochs
epochs = 100
optimizer = optim.Adam(model.parameters(), lr=0.01) # Reinitialize the optimizer
for epoch in range(1,epochs+1):
    optimizer.zero_grad() # Zero the gradients
    y_pred = model(X_train_tensor) # Forward pass
    loss = criterion(y_pred, y_train_tensor) # Compute the loss
    loss.backward() # Backward pass
    optimizer.step() # Update the weights
    if epoch % 10 == 0:
        print(f'Epoch: {epoch}, Loss: {loss.detach().item()}')

Epoch: 10, Loss: 1.4502893686294556
Epoch: 20, Loss: 0.8866540789604187
Epoch: 30, Loss: 0.5856936573982239
Epoch: 40, Loss: 0.5078949332237244
Epoch: 50, Loss: 0.48694950342178345
Epoch: 60, Loss: 0.47141730785369873
Epoch: 70, Loss: 0.462691992521286
Epoch: 80, Loss: 0.45946013927459717
Epoch: 90, Loss: 0.4582115113735199
Epoch: 100, Loss: 0.45767083764076233
